In [1]:
import os
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
client = OpenAI(api_key=openai_api_key)

In [6]:
file_path = './11_version_9.jsonl'

In [7]:
for job in client.fine_tuning.jobs.list():
    print(job)

FineTuningJob(id='ftjob-tQHGeBFpQMTC68MhOyl1GCU1', created_at=1730310435, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Invalid file format. Input file file-DprFZ7K9bdwvALFY2mVdOxCS is in the prompt-completion format, but the specified model gpt-3.5-turbo-0125 is a chat model and requires chat-formatted data. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for details.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-tkmi69gRHiQsUbafMz0TPIlN', result_files=[], seed=824383575, status='failed', trained_tokens=None, training_file='file-DprFZ7K9bdwvALFY2mVdOxCS', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
FineTuningJob(id='ftjob-GxfIsWvE9nKLUc0nhtCjzJy1', created_at=1730

In [8]:
uploaded_dataset = client.files.create(
    file=open(file_path, 'rb'),
    purpose='fine-tune'
)
uploaded_dataset

FileObject(id='file-YG1WjBAProvE7UDQmYzanQgp', bytes=51777, created_at=1730311981, filename='11_version_9.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [10]:
print(uploaded_dataset.id)
type(uploaded_dataset.id)

file-YG1WjBAProvE7UDQmYzanQgp


str

In [11]:
training_detail = client.fine_tuning.jobs.create(
    training_file=uploaded_dataset.id,
    model='gpt-4o-mini-2024-07-18'
)

In [12]:
print(training_detail.id)
type(training_detail.id)

ftjob-Tbdf6OsN1uHPpiW7K2XnjZy3


str

In [33]:
for job in client.fine_tuning.jobs.list_events(fine_tuning_job_id=training_detail.id):
    print(job)

FineTuningJobEvent(id='ftevent-lV2bC23iw3Hj8mVhgPcwfP4J', created_at=1730312594, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message')
FineTuningJobEvent(id='ftevent-7IIBl8ncPJXzQ5eO3PLlUgcR', created_at=1730312587, level='info', message='New fine-tuned model created', object='fine_tuning.job.event', data={}, type='message')
FineTuningJobEvent(id='ftevent-L0eWM6C0VThD5osss3CS0Dnv', created_at=1730312587, level='info', message='Checkpoint created at step 230', object='fine_tuning.job.event', data={}, type='message')
FineTuningJobEvent(id='ftevent-oINPw3spgqwwNKEd8wszc1q7', created_at=1730312587, level='info', message='Checkpoint created at step 115', object='fine_tuning.job.event', data={}, type='message')
FineTuningJobEvent(id='ftevent-avrLWW763MzTpcqmeTEL4dmn', created_at=1730312572, level='info', message='Step 345/345: training loss=0.16', object='fine_tuning.job.event', data={'step': 345, 'train_loss': 0.1618085056543350

In [58]:
client.fine_tuning.jobs.list_events(fine_tuning_job_id=training_detail.id, limit=1).data[0].message

'The job has successfully completed'

In [43]:
for job in client.fine_tuning.jobs.list():
    if job.id == training_detail.id:
        model_id = job.fine_tuned_model

print(model_id)

ft:gpt-4o-mini-2024-07-18:personal::AO7dT1L1


In [59]:
completion = client.chat.completions.create(
  model=model_id,
  messages=[
    {"role": "system", "content": "You are an assistant tasked with providing engaging and relevant responses based on the given conversation context. Respond thoughtfully and appropriately to user inputs."},
    {"role": "user", "content": "엄마 점심 먹었어?"}
  ]
)
print(completion.choices[0].message.content)

아직 안 먹었어, 뭘 먹을까 고민 중이야.
